In [1]:
import pandas as pd
import numpy as np
import datetime
from sklearn.model_selection import train_test_split

#%matplotlib inline

# Cargo CSV

In [125]:
test_final = pd.read_csv('../data/test_final_100k.csv')
test_final = test_final.drop(columns='id')
test_final.head()

,idaviso,idpostulante
0,739260,6M9ZQR
1,739260,6v1xdL
2,739260,ezRKm9
3,758580,1Q35ej
4,758580,EAN4J6


In [3]:
postulantes_educacion = pd.read_csv('../data_sin_duplicados/fiuba_1_postulantes_educacion.csv')
postulantes_educacion = postulantes_educacion.drop(columns='Unnamed: 0')
postulantes_educacion.head()

,idpostulante,nombre,estado
0,8BkL,Universitario,En Curso
1,1d2B,Universitario,En Curso
2,6M3jr,Universitario,En Curso
3,Z85B5,Universitario,En Curso
4,1QLLO,Universitario,En Curso


In [4]:
postulantes_genero_y_edad = pd.read_csv('../data_sin_duplicados/fiuba_2_postulantes_genero_y_edad.csv')
postulantes_genero_y_edad = postulantes_genero_y_edad.drop(columns='Unnamed: 0')
postulantes_genero_y_edad.head()

,idpostulante,fechanacimiento,sexo
0,NM5M,1970-12-03,FEM
1,5awk,1962-12-04,FEM
2,ZaO5,1978-08-10,FEM
3,NdJl,1969-05-09,MASC
4,eo2p,1981-02-16,MASC


vistas = pd.read_csv('../data_sin_duplicados/fiuba_3_vistas.csv')
vistas = vistas.drop(columns='Unnamed: 0')
vistas.head()

In [5]:
postulaciones = pd.read_csv('../data_sin_duplicados/fiuba_4_postulaciones.csv')
postulaciones = postulaciones.drop(columns='Unnamed: 0')
postulaciones.head()

,idaviso,idpostulante,fechapostulacion
0,1112257047,NM5M,2018-01-15 16:22:34
1,1111920714,NM5M,2018-02-06 09:04:50
2,1112346945,NM5M,2018-02-22 09:04:47
3,1112345547,NM5M,2018-02-22 09:04:59
4,1112237522,5awk,2018-01-25 18:55:03


avisos_online = pd.read_csv('../data_sin_duplicados/fiuba_5_avisos_online.csv')
avisos_online = avisos_online.drop(columns='Unnamed: 0')
avisos_online.head()

In [6]:
avisos_detalle = pd.read_csv('../data_sin_duplicados/fiuba_6_avisos_detalle.csv')
avisos_detalle = avisos_detalle.drop(columns='Unnamed: 0')
avisos_detalle.head()

,idaviso,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1000872556,1,Operarios de Planta - Rubro Electrodomésticos,<p><strong>OPERARIOS DE PLANTA</strong></p><p>...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Producción,ELECTRO OUTLET SRL
1,9240880,1,"Productores Asesores Independiente, para venta...",Agente\r\noficial Selecciona:</span></strong><...,Capital Federal,Buenos Aires,NaN,Full-time,Jefe / Supervisor / Responsable,Comercial,Agencia Oficial Alejandro Arizaga
2,1110185164,1,Administrativo de Recepción,<p>Centro Médico Accord se encuentra en la bús...,Gran Buenos Aires,NaN,NaN,Full-time,Junior,Administración,Unión Personal
3,1110513885,1,Distribuidor domiciliario con moto (SAN MIGUEL),<p>Importante correo privado ubicado en SAN MI...,Gran Buenos Aires,NaN,NaN,Full-time,Junior,Distribución,Asoko Tempo SA
4,1111034024,1,Vendedores para venta de medicina pre paga y o...,<p> </p><p><strong>Comercializadora incorpora ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,JELS SRL


### Agrego edad

In [7]:
# Agrego la edad de los postulantes
postulantes_genero_y_edad['fechanacimiento'] = pd.to_datetime(postulantes_genero_y_edad['fechanacimiento'],errors='coerce')
postulantes_genero_y_edad['edad'] = datetime.datetime.now().year - postulantes_genero_y_edad['fechanacimiento'].dt.year

In [8]:
postulantes_genero_y_edad.head()

,idpostulante,fechanacimiento,sexo,edad
0,NM5M,1970-12-03,FEM,48.0
1,5awk,1962-12-04,FEM,56.0
2,ZaO5,1978-08-10,FEM,40.0
3,NdJl,1969-05-09,MASC,49.0
4,eo2p,1981-02-16,MASC,37.0


In [9]:
# Despues veo que hago con los que no tienen edad
postulantes_genero_y_edad.isna().sum()

idpostulante           0
fechanacimiento    26096
sexo                   0
edad               26096
dtype: int64

In [10]:
# Separa la gente postulada, luego los busco en el DF que junta todos los postulantes
# y me quedo con los opuestos, los no postulados.
# esto me va a servir para samplear
# GENTE NO POSTULADA
gente_no_postulada = postulantes_genero_y_edad[-postulantes_genero_y_edad['idpostulante'].\
                                               isin(postulaciones['idpostulante'].value_counts().index)]
gente_no_postulada = gente_no_postulada.reset_index(drop=True)

In [11]:
gente_no_postulada['sexo'].value_counts()

FEM           71992
MASC          67819
NO_DECLARA    15707
0.0               6
Name: sexo, dtype: int64

In [12]:
# Pocos datos con sexo invalido, los desecho ya que son los no postulados
gente_no_postulada = gente_no_postulada.drop(gente_no_postulada[gente_no_postulada['sexo'] == '0.0'].index)

In [13]:
# GENTE POSTULADA
gente_postulada = postulantes_genero_y_edad[postulantes_genero_y_edad['idpostulante'].\
                                               isin(postulaciones['idpostulante'].value_counts().index)]
gente_postulada = gente_postulada.reset_index(drop=True)

In [14]:
gente_postulada['sexo'].value_counts()

FEM           179213
MASC          160034
NO_DECLARA      9635
0.0                1
Name: sexo, dtype: int64

In [15]:
# Reemplazo el sexo invalido por NO_DECLARA
gente_postulada.loc[gente_postulada['sexo'] == '0.0','sexo'] = 'NO_DECLARA'

In [16]:
# Dropeo los datos invalidos de los no postulados (se podria buscar alguna forma de salvarlos)
gente_no_postulada = gente_no_postulada.dropna()
gente_no_postulada = gente_no_postulada.reset_index(drop=True)
gente_no_postulada['edad'] = gente_no_postulada['edad'].astype('int32')

In [17]:
# En un principio los relleno con la media de los postulantes
gente_postulada.loc[gente_postulada['edad'].isna(),'edad'] = int(postulantes_genero_y_edad['edad'].mean())
gente_postulada['edad'] = gente_postulada['edad'].astype('int32')

In [18]:
gente_postulada.isna().sum()

idpostulante           0
fechanacimiento    10074
sexo                   0
edad                   0
dtype: int64

# Funciones de utilidad

In [19]:
# fijarse de balancear el sample con 50% de postulados y no postulados

# Generador de sample
# el id tiene que estar en la columna 0
# Recibe los DFs de idpostulante e idaviso
def generador_de_sample_df(idaviso, idpostulante, n_sample, ciclos):
    idpostulante = idpostulante.drop(columns=idpostulante.columns[1:])
    idaviso = idaviso.drop(columns=idaviso.columns[1:])
    idpostulante['tem'] = 1
    idaviso['tem'] = 1
    sample_append = pd.DataFrame(columns=[idpostulante.columns[0],'tem',idaviso.columns[0]])
    for i in range(0,ciclos):
        sample_idpostulante = idpostulante.sample(n=n_sample,random_state=i)
        sample_idaviso = idaviso.sample(n=n_sample,random_state=i)
        sample_merge = sample_idaviso.merge(sample_idpostulante)
        sample_append = sample_append.append(sample_merge)
    sample_append = sample_append.drop(columns='tem')
    sample_append = sample_append.reset_index(drop=True)
    return sample_append

In [20]:
# Recibe las series de idpostulante e idaviso
def generador_de_sample_serie(idaviso, idpostulante, n_sample, ciclos):
    idpostulante = pd.DataFrame(idpostulante)
    idaviso = pd.DataFrame(idaviso)
    idpostulante['tem'] = 1
    idaviso['tem'] = 1
    sample_append = pd.DataFrame(columns=[idpostulante.columns[0],'tem',idaviso.columns[0]])
    for i in range(0,ciclos):
        sample_idpostulante = idpostulante.sample(n=n_sample,random_state=i)
        sample_idaviso = idaviso.sample(n=n_sample,random_state=i)
        sample_merge = sample_idaviso.merge(sample_idpostulante)
        sample_append = sample_append.append(sample_merge)
    sample_append = sample_append.drop(columns='tem')
    sample_append = sample_append.reset_index(drop=True)
    return sample_append

In [21]:
def borrar_tuplas_test_del_sample(sample,test_final):
    df_return = sample.merge(test_final,how='left',on=['idaviso','idpostulante'],indicator=True)
    df_return = df_return[df_return['_merge'] == 'left_only']
    df_return = df_return.drop(columns='_merge')
    return df_return

In [22]:
def oneHotEncoding(df, columna):
    one_hot = pd.get_dummies(df[columna])
    df.drop(columna, axis=1, inplace=True)
    df = pd.concat([df, one_hot], axis=1)
    return df

In [23]:
# la siguiente funcion tiene en cuenta los estados maximos de los diferentes estudios
def educacion_ponderado_por_estado(postulantes_educacion):
    dic = {'Graduado': 3,'En Curso': 2,'Abandonado': 1}
    postulantes_educacion['estado'] = postulantes_educacion['estado'].map(dic)
    postulantes_educacion = postulantes_educacion.groupby(['idpostulante','nombre']).max().unstack(fill_value=0)
    postulantes_educacion.columns = postulantes_educacion.columns.droplevel()
    postulantes_educacion = postulantes_educacion.reset_index()
    postulantes_educacion = postulantes_educacion.rename_axis('',axis='columns')
    return postulantes_educacion

In [24]:
# la siguiente funcion tiene en cuenta solo el estado maximo sin diferenciar estudios
def educacion_ponderado_por_estado_maximo(postulantes_educacion):
    dic = {'Graduado': 3,'En Curso': 2,'Abandonado': 1}
    postulantes_educacion['estado'] = postulantes_educacion['estado'].map(dic)
    postulantes_educacion = postulantes_educacion.groupby('idpostulante').agg({'estado':'min'})
    postulantes_educacion = postulantes_educacion.reset_index()
    return postulantes_educacion

In [25]:
postulantes_educacion = educacion_ponderado_por_estado_maximo(postulantes_educacion)

In [26]:
postulantes_educacion.head()

,idpostulante,estado
0,0z5Dmrd,3
1,0z5JW1r,1
2,0z5VvGv,3
3,0zB01pE,1
4,0zB026d,3


# Funciones feature

In [27]:
def feature_postulante_se_postulo_al_aviso(tupla_idaviso_idpostulante,postulaciones):
    df_return = tupla_idaviso_idpostulante.merge(postulaciones,on=['idaviso','idpostulante'],how='left')
    return df_return

In [28]:
def feature_genero_edad(tupla_idaviso_idpostulante,postulantes_genero_y_edad):
    df_return = tupla_idaviso_idpostulante.merge(postulantes_genero_y_edad,on='idpostulante',how='left')
    return df_return

# Gente postulada

In [29]:
tupla_postulada = postulaciones.drop(columns='fechapostulacion')
tupla_postulada['y'] = 1
tupla_postulada = feature_genero_edad(tupla_postulada,gente_postulada)
tupla_postulada = tupla_postulada.drop(columns='fechanacimiento')
tupla_postulada = oneHotEncoding(tupla_postulada,'sexo')

In [30]:
tupla_postulada.head()

,idaviso,idpostulante,y,edad,FEM,MASC,NO_DECLARA
0,1112257047,NM5M,1,48,1,0,0
1,1111920714,NM5M,1,48,1,0,0
2,1112346945,NM5M,1,48,1,0,0
3,1112345547,NM5M,1,48,1,0,0
4,1112237522,5awk,1,56,1,0,0


In [31]:
tupla_postulada = tupla_postulada.merge(postulantes_educacion,on='idpostulante',how='left')

In [32]:
tupla_postulada['estado'].isna().sum()

204726

In [33]:
tupla_postulada.loc[tupla_postulada['estado'].isna(),'estado'] = 0
tupla_postulada['estado'] = tupla_postulada['estado'].astype('int32')

In [34]:
tupla_postulada.head()

,idaviso,idpostulante,y,edad,FEM,MASC,NO_DECLARA,estado
0,1112257047,NM5M,1,48,1,0,0,3
1,1111920714,NM5M,1,48,1,0,0,3
2,1112346945,NM5M,1,48,1,0,0,3
3,1112345547,NM5M,1,48,1,0,0,3
4,1112237522,5awk,1,56,1,0,0,3


# Gente no postulada

In [35]:
sample = generador_de_sample_serie(avisos_detalle['idaviso'],gente_no_postulada['idpostulante'],100,100)
sample['y'] = 0
sample = feature_genero_edad(sample,gente_no_postulada)
sample = sample.drop(columns='fechanacimiento')
sample = oneHotEncoding(sample,'sexo')

In [36]:
sample.head()

,idaviso,idpostulante,y,edad,FEM,MASC,NO_DECLARA
0,1112390041,GNZ8JWK,0,55,0,1,0
1,1112390041,ZDlz5E1,0,21,1,0,0
2,1112390041,a0N4q,0,43,1,0,0
3,1112390041,ep41j9,0,32,0,1,0
4,1112390041,awOL1m,0,34,1,0,0


In [37]:
sample = sample.merge(postulantes_educacion,on='idpostulante',how='left')

In [38]:
sample['estado'].isna().sum()

134100

In [39]:
sample.loc[sample['estado'].isna(),'estado'] = 0
sample['estado'] = sample['estado'].astype('int32')

In [40]:
sample.head()

,idaviso,idpostulante,y,edad,FEM,MASC,NO_DECLARA,estado
0,1112390041,GNZ8JWK,0,55,0,1,0,0
1,1112390041,ZDlz5E1,0,21,1,0,0,2
2,1112390041,a0N4q,0,43,1,0,0,3
3,1112390041,ep41j9,0,32,0,1,0,2
4,1112390041,awOL1m,0,34,1,0,0,3


# Test Final

In [66]:
test_final = feature_genero_edad(test_final,gente_postulada.append(gente_no_postulada))

In [67]:
# Veo si hay datos nulos
test_final.isna().sum()

idaviso               0
idpostulante          0
fechanacimiento    2838
sexo               2278
edad               2278
dtype: int64

In [68]:
# Salvo datos nulos
test_final.loc[test_final['sexo'].isna(),'sexo'] = 'NO_DECLARA'
test_final.loc[test_final['edad'].isna(),'edad'] = int(postulantes_genero_y_edad['edad'].mean())

In [69]:
test_final.isna().sum()

idaviso               0
idpostulante          0
fechanacimiento    2838
sexo                  0
edad                  0
dtype: int64

In [70]:
test_final.head()

,idaviso,idpostulante,fechanacimiento,sexo,edad
0,739260,6M9ZQR,1976-03-29,FEM,42.0
1,739260,6v1xdL,1987-06-27,MASC,31.0
2,739260,ezRKm9,1982-02-17,FEM,36.0
3,758580,1Q35ej,1949-12-24,MASC,69.0
4,758580,EAN4J6,1986-05-27,FEM,32.0


In [71]:
#test_final = test_final.drop(columns=['idaviso','idpostulante','fechanacimiento'])
test_final['edad'] = test_final['edad'].astype('int32')
test_final = test_final.drop(columns='fechanacimiento')
test_final = oneHotEncoding(test_final,'sexo')

In [72]:
test_final.head()

,idaviso,idpostulante,edad,FEM,MASC,NO_DECLARA
0,739260,6M9ZQR,42,1,0,0
1,739260,6v1xdL,31,0,1,0
2,739260,ezRKm9,36,1,0,0
3,758580,1Q35ej,69,0,1,0
4,758580,EAN4J6,32,1,0,0


In [73]:
test_final = test_final.merge(postulantes_educacion,on='idpostulante',how='left')

In [76]:
test_final['estado'].isna().sum()

0

In [75]:
test_final.loc[test_final['estado'].isna(),'estado'] = 0
test_final['estado'] = test_final['estado'].astype('int32')

In [79]:
test_final.head()

,idaviso,idpostulante,edad,FEM,MASC,NO_DECLARA,estado
0,739260,6M9ZQR,42,1,0,0,3
1,739260,6v1xdL,31,0,1,0,2
2,739260,ezRKm9,36,1,0,0,3
3,758580,1Q35ej,69,0,1,0,3
4,758580,EAN4J6,32,1,0,0,1


In [80]:
test_final = test_final.drop(columns=['idaviso','idpostulante'])

# ML

In [41]:
# Metodo para calcular el error de la competencia
from sklearn.metrics import roc_auc_score
# Metodos de clasificacion
from sklearn.neighbors import KNeighborsClassifier

In [42]:
sample = sample.append(tupla_postulada.sample(n=1000000,random_state=0))

In [43]:
sample.head()

,idaviso,idpostulante,y,edad,FEM,MASC,NO_DECLARA,estado
0,1112390041,GNZ8JWK,0,55,0,1,0,0
1,1112390041,ZDlz5E1,0,21,1,0,0,2
2,1112390041,a0N4q,0,43,1,0,0,3
3,1112390041,ep41j9,0,32,0,1,0,2
4,1112390041,awOL1m,0,34,1,0,0,3


In [44]:
sample.tail()

,idaviso,idpostulante,y,edad,FEM,MASC,NO_DECLARA,estado
4019134,1112457433,RzMAmXo,1,24,1,0,0,2
1441298,1112306596,ekOJ0L9,1,31,0,1,0,3
1290650,1112262555,PmJExxE,1,29,1,0,0,2
4598828,1112296411,4rPXrkM,1,25,0,1,0,0
6441257,1112355579,xkPYo6z,1,42,0,1,0,3


In [45]:
sample = sample.drop(columns=['idaviso','idpostulante'])

In [46]:
len(sample)

2000000

In [47]:
sample.head()

,y,edad,FEM,MASC,NO_DECLARA,estado
0,0,55,0,1,0,0
1,0,21,1,0,0,2
2,0,43,1,0,0,3
3,0,32,0,1,0,2
4,0,34,1,0,0,3


In [122]:
# Divido el set
X_train, X_test, y_train, y_test = train_test_split(sample.drop(columns='y'),sample['y'],random_state=0,test_size=0.01)

In [123]:
y_train.value_counts()

1    990048
0    989952
Name: y, dtype: int64

In [124]:
y_test.value_counts()

0    10048
1     9952
Name: y, dtype: int64

# Perceptron

In [57]:
from sklearn.linear_model import Perceptron

In [58]:
model = Perceptron()
model.fit(X_train,y_train)
predict = model.predict(X_test)

/usr/local/lib/python2.7/dist-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [59]:
roc_auc_score(y_test,predict)

0.5394565890798124

# KNN

In [ ]:
model = KNeighborsClassifier(n_neighbors=3)
model.fit(X_train,y_train)
predict = model.predict(X_test)

In [ ]:
roc_auc_score(y_test,predict)

# Random Forest

In [99]:
from sklearn.ensemble import RandomForestClassifier

In [100]:
model = RandomForestClassifier(n_estimators=100,random_state=0, n_jobs=-1)
model.fit(X_train, y_train)
predict = model.predict(X_test)

In [101]:
roc_auc_score(y_test,predict)

0.6181468821041637

# Gradient Boosting Classifier

In [63]:
from sklearn.ensemble import GradientBoostingClassifier

In [64]:
model = GradientBoostingClassifier(n_estimators=150)
model.fit(X_train, y_train)
predict = model.predict(X_test)

In [65]:
roc_auc_score(y_test,predict)

0.6121691443770865

In [97]:
predict = pd.DataFrame(predict,columns=['sepostulo'])

In [98]:
predict.to_csv('Random_Forest_Modelo_2.csv')